In [1]:
import random

RandomListOfIntegers = [random.randint(0, 99999) for iter in range(5)]


In [2]:
RandomListOfIntegers

[88476, 39213, 23551, 11749, 41655]

In [ ]:
import numpy as np
import torch
from torch.autograd import Variable
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from glob import glob
import torch.backends.cudnn as cudnn
from modules.vgg import vgg16_bn
from modules.resnet import resnet50, resnet101
import matplotlib.cm
from matplotlib.cm import ScalarMappable
import argparse
import easydict
import os
import pickle

from utils.config import *
from utils.count_model import *
from datasets.cifar100 import *


def compute_pred(output):
    pred = output.data.max(1, keepdim=True)[1]  # get the index of the max log-probability
    print('Pred cls : '+str(pred))
    T = pred.squeeze().cpu().numpy()
    T = np.expand_dims(T, 0)
    T = (T[:, np.newaxis] == np.arange(100)) * 1.0
    T = torch.from_numpy(T).type(torch.FloatTensor)
    Tt = Variable(T).cuda()
    return Tt


for i, seed in enumerate(RandomListOfIntegers):
    config = easydict.EasyDict()
    config.exp_name = "vgg16_exp_cifar100_0"
    config.log_dir = os.path.join("experiments", config.exp_name, "logs/")

    config.load_file = os.path.join("experiments", "vgg16_exp_cifar100_0","checkpoints", "checkpoint.pth")
    config.cuda = True
    config.gpu_device = 0
    config.seed = seed
    config.milestones = [10, 20,30,40]
    config.gamma = 0.9
    config.img_size = 32
    config.num_classes = 100
    config.data_mode = "download"
    config.data_loader_workers = 4
    config.pin_memory = True
    config.async_loading = True
    config.batch_size = 1
    config.async_loading = True
    config.max_epoch = 20
    config.images_per_class = 3

    torch.cuda.init()

    cudnn.benchmark = True
    # define data loader
    data_loader = Selected_Cifar100DataLoader(config)
    # define model
    model = vgg16_bn(input_shape=config.img_size, num_classes=config.num_classes, data_set='Cifar100')
    model.load_checkpoint(config.load_file)
    model.init_graph(config)

    # method = LRP or RAP
    method = 'LRP'

    is_cuda = torch.cuda.is_available()
    if is_cuda and not config.cuda:
        print("WARNING: You have a CUDA device, so you should probably enable CUDA")
    cuda = is_cuda & config.cuda
    manual_seed = config.seed
    if cuda:
        device = torch.device("cuda")
        torch.cuda.manual_seed(manual_seed)
        torch.cuda.set_device(config.gpu_device)

    print(model)
    model = model.to(device)

    # evaluate
    model.eval()

    for idx, (x, y) in enumerate(data_loader.valid_loader):

        if idx in list(range(2,300,3)):
            x, y = x.cuda(non_blocking=True), y.cuda(non_blocking=True)
            output = model(x)
            T = compute_pred(output)
            Res = model.relprop(R = output * T)
            model.record_relevance()
            model.relevance_generating_shortest_igraph()
            print('relevance_generating_shortest_igraph',y, idx)
        else:
            x, y = x.cuda(non_blocking=True), y.cuda(non_blocking=True)
            output = model(x)
            T = compute_pred(output)
            Res = model.relprop(R = output * T)
            model.record_relevance()
            print(y, idx)



    print('Done')
    record_stayed_nodes = np.array(model.record_stayed_nodes)

    with open('Record_relevance_VGG16_Transform_3class_top20_{}.txt'.format(i), 'wb') as f:
         pickle.dump(record_stayed_nodes, f)


Files already downloaded and verified
32 100 Cifar100
Loading checkpoint 'experiments\vgg16_exp_cifar100_0\checkpoints\checkpoint.pth'
Checkpoint loaded successfully at (epoch 240) at (iteration 93840)

Program will run on *****GPU-CUDA*****

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), pad